<a href="https://colab.research.google.com/github/rarabzad/CaSR-v3.1-downloader/blob/main/CaSR_v3_1_streamlit_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Run the CaSR v3.1 Downloader Streamlit App on Google Colab

You can easily run the **CaSR v3.1 Downloader** Streamlit web app directly from **Google Colab**, without needing to host it locally or deploy it permanently.

This setup uses **Cloudflare Tunnel** to expose your Colab environment securely to the web, allowing you to access your Streamlit app in a browser tab.

---

1. Open a new notebook in **Google Colab**:
   [https://colab.research.google.com](https://colab.research.google.com)

2. Copy and paste the entire code block below into a single cell.

3. Run the cell (Shift + Enter).
   Wait about **30–60 seconds** until you see a link like:

   ```
   Streamlit app is live: https://example-xyz.trycloudflare.com
   ```

   The app should open automatically in a new tab — if not, just click the link.

### ⚙️ Notes

* **Runtime:** Keep the Colab session active — closing the tab or timing out will stop the app.
* **Cloudflare tunnel** is temporary — each time you restart, a new URL is generated.
* This setup doesn’t require a Cloudflare account, but for long-term hosting you can create a named tunnel.
* Works best in **Chrome** or **Edge** (some popup blockers may prevent auto-tab opening).


In [ ]:
# Install dependencies
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
!pip install -q streamlit

# Clean and clone repo
!rm -rf CaSR-v3.1-downloader
!git clone https://github.com/rarabzad/CaSR-v3.1-downloader.git
%cd CaSR-v3.1-downloader
!pip install -q -r requirements.txt

import re, subprocess, threading, time, webbrowser
from IPython.display import display, HTML, Javascript

public_url = None

def run_cloudflared():
    global public_url
    proc = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", "http://localhost:8501", "--no-autoupdate"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
    )
    for line in proc.stdout:
        match = re.search(r"(https://[0-9a-zA-Z-]+\.trycloudflare\.com)", line)
        if match:
            public_url = match.group(1)
            display(HTML(f"<h3> Streamlit app is live: "
                         f"<a href='{public_url}' target='_blank'>{public_url}</a></h3>"))
            webbrowser.open_new_tab(public_url)
            break
        print(line, end="")

# Start Cloudflare tunnel in background
threading.Thread(target=run_cloudflared, daemon=True).start()
time.sleep(5)

# Start Streamlit app
!streamlit run app.py --server.port 8501

if public_url:
    display(HTML(f"<h3> App is live: <a href='{public_url}' target='_blank'>{public_url}</a></h3>"))
    display(Javascript(f"window.open('{public_url}');"))
else:
    print("No Cloudflare URL detected yet — wait a few seconds.")